![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [61]:
import pandas as pd
import numpy as np

## Client

In [62]:
# list of the columns we want to use
cols = ['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']

# import csv with the columns necessary for clients
client = pd.read_csv('bank_marketing.csv', usecols=cols)

In [63]:
client = client.replace({'education':{'unknown': np.NaN}, 
                         'credit_default':{'unknown': np.NaN},
                         'mortgage': {'unknown': np.NaN}
                        })

In [64]:
#changing the columns creadit_default and mortgage type to bool
client[['mortgage', 'credit_default']] = client[['mortgage','credit_default']].astype(bool)

In [65]:
client['education'] = client['education'].str.replace('.', '_')

In [66]:
client['job'] = client['job'].str.replace('.', '_')

## Campaign

In [67]:
# list of the columns we want to use
cols = ['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'month', 'day']

# import csv with the columns necessary for campaign
campaign = pd.read_csv('bank_marketing.csv', 
                        usecols=cols,
                        dtype={'campaign_outcome':bool},
                        true_values=['yes'],
                        false_values=['no'],
                        parse_dates=[['month', 'day']])

In [68]:
campaign = campaign.replace({'previous_outcome':{'nonexistent': np.NaN}, 
                        })

In [69]:
campaign['previous_outcome'] = campaign['previous_outcome'].astype(bool)

In [70]:
# Adding the year to the month_day column and turn it into a datetime format
campaign['last_contact_date'] = pd.to_datetime("2022 " + campaign['month_day'], format='%Y %b %d')
campaign.drop('month_day', axis=1, inplace=True)

## Economics

In [71]:
# list of the columns we want to use
cols = ['client_id', 'cons_price_idx', 'euribor_three_months']

# import csv with the columns necessary for economics
economics = pd.read_csv('bank_marketing.csv', usecols=cols)

In [72]:
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)